In [434]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.optimize as opt
from sklearn.model_selection import train_test_split
import random as rnd

In [435]:
data = pd.read_excel("C:/Users/Niku/Documents/dataset/arrays.xlsx")
data.shape

(1992, 36)

In [436]:
X = data[0:1500]
X = np.array(X)
y = X[:, 35]
#X = X[:, 0:4]
X.shape
#y = np.matrix(y)
y = np.array(y)
y[y == 0] = -1
y1 = np.matrix(y)
y.shape, X.shape, y1.shape

((1500L,), (1500L, 36L), (1L, 1500L))

In [437]:
def kernel_linear(x1, x2):
    return np.dot(x1, x2.T)
def kernel_quadratic(x1, x2):
    return (np.dot(x1, x2.T) ** 2)

In [438]:
def rnd_int(a,b,z):
    i = z
    while i == z:
        i = rnd.randint(a,b)
    return i

In [439]:
def predict(X):
    return h(X, w, b)

In [440]:
def calc_b(X, y, w):
    b_tmp = y - np.dot(w.T, X.T)
    return np.mean(b_tmp)

In [441]:
def calc_w(alpha, y, X):
    return np.dot(alpha * y, X)

In [442]:
def h(X, w, b):
    return np.sign(np.dot(w.T, X.T) + b).astype(int)

In [443]:
def E(x_k, y_k, w, b):
    return h(x_k, w, b) - y_k

In [444]:
 def compute_L_H(C, alpha_prime_j, alpha_prime_i, y_j, y_i):
    if(y_i != y_j):
        return (max(0, alpha_prime_j - alpha_prime_i), min(C, C - alpha_prime_i + alpha_prime_j))
    else:
        return (max(0, alpha_prime_i + alpha_prime_j - C), min(C, alpha_prime_i + alpha_prime_j))

In [445]:
max_iter=10000
kernel_type='linear'
C=1.0
epsilon=0.001

In [446]:
def fit(X, y):
    # Initialization
    n, d = X.shape[0], X.shape[1]
    alpha = np.zeros((n))
    #alpha = np.array(alpha)
    #print (alpha.shape)
    count = 0
    while True:
        count += 1
        alpha_prev = np.copy(alpha)
        for j in range(0, n):
            i = rnd_int(0, n-1, j) # Get random int i~=j
            x_i, x_j, y_i, y_j = X[i,:], X[j,:], y[i], y[j]
            #k_ij = kernel_quadratic(x_i, x_i) + kernel_quadratic(x_j, x_j) - 2 * kernel_quadratic(x_i, x_j)
            k_ij = kernel_linear(x_i, x_i) + kernel_linear(x_j, x_j) - 2 * kernel_linear(x_i, x_j)
            if k_ij == 0:
                continue
            alpha_prime_j, alpha_prime_i = alpha[j], alpha[i]
            (L, H) = compute_L_H(C, alpha_prime_j, alpha_prime_i, y_j, y_i)

                # Compute model parameters
            w = calc_w(alpha, y, X)
            b = calc_b(X, y, w)
            
                # Compute E_i, E_j
            E_i = E(x_i, y_i, w, b)
            E_j = E(x_j, y_j, w, b)

                # Set new alpha values
            alpha[j] = alpha_prime_j + float(y_j * (E_i - E_j))/k_ij
            alpha[j] = max(alpha[j], L)
            alpha[j] = min(alpha[j], H)

            alpha[i] = alpha_prime_i + y_i*y_j * (alpha_prime_j - alpha[j])

            # Check convergence
        diff = np.linalg.norm(alpha - alpha_prev)
        if diff < epsilon:
            break

        if count >= max_iter:
            print("Iteration number exceeded the max of %d iterations" % (max_iter))
            return
        # Compute final model parameters
    b = calc_b(X, y, w)
    if kernel_type == 'linear':
        w = calc_w(alpha, y, X)
    # Get support vectors
    alpha_idx = np.where(alpha > 0)[0]
    support_vectors = X[alpha_idx, :]
    return support_vectors, count

In [447]:
fit(X,y)

(array([[ 0.33152174,  0.00827558,  0.95812274,  0.05937663,  0.00767048,
          0.12906463,  0.19499427,  0.09901662,  0.30365878,  1.        ,
          0.32433132,  0.72851863,  0.42875105,  0.08516276,  0.46111772,
          0.25679842,  0.06932337,  0.10887503,  0.16249825,  0.3925    ,
          0.04931142,  0.62055138,  0.15909091,  0.89097132,  0.00992146,
          0.87945707,  0.0472215 ,  0.01615074,  0.85559748,  0.01311892,
          0.53770087,  0.        ,  0.        ,  0.04922444,  0.07067918,
          0.        ],
        [ 0.47282609,  0.19623461,  0.52521919,  0.36235417,  0.11148471,
          0.23243677,  0.45542287,  0.2449983 ,  0.04832714,  1.        ,
          0.15588716,  0.90406529,  0.08284437,  0.24224073,  0.2979786 ,
          0.1050719 ,  0.18742217,  0.11269873,  0.19197988,  0.21361111,
          0.09684585,  0.84177109,  0.3992629 ,  0.78031291,  0.06986358,
          0.707099  ,  0.11244456,  0.20003365,  0.22188679,  0.06855692,
          0.585